In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

if __name__ == '__main__':
    s = '1011'
    print('Secret string s =', s)
    qc = bernstein_vazirani_circuit(s)
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print('✅ Successfully recovered secret string s')
    else:
        print('⚠️ Measured string differs from s (noise or error).')


Secret string s = 1011
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured bitstring (input register): 1101
⚠️ Measured string differs from s (noise or error).


In [3]:
# ============================================================
# Bernstein–Vazirani (Qiskit 2.x) — Complete, Robust Notebook
# ============================================================
# ✓ Change secret s and verify measured output matches s
# ✓ Add constant bit b (f(x) = s·x ⊕ b) and show it only affects ancilla/global phase
# ✓ Compare ideal vs noisy runs (custom Aer NoiseModel)
# ✓ Optional: run on IBM backend via qiskit_ibm_runtime (safe try/except)
# ✓ Clear section prints + plots (no seaborn; pure matplotlib + qiskit.plot_histogram)
# ------------------------------------------------------------

# ---------- Install ----------
!pip install -q qiskit qiskit_aer qiskit_ibm_runtime

# ---------- Imports ----------
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt

# IBM Runtime (optional)
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
    IBM_RUNTIME_AVAILABLE = True
except Exception:
    IBM_RUNTIME_AVAILABLE = False


# ============================================================
# 0) Utility: Pretty headings (so your cells look like notes)
# ============================================================
def h1(title):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))

def h2(title):
    print("\n" + title)
    print("-"*len(title))


# ============================================================
# 1) Oracles
# ============================================================
def bv_oracle_with_b(qc, inputs, ancilla, s: str, b: int = 0):
    """
    Computational-oracle style: y -> y ⊕ f(x), where f(x) = s·x ⊕ b
    Implemented by:
      - For each i with s[i] == '1': CX(inputs[i], ancilla)
      - If b == 1: X(ancilla)
    NOTE: In the actual BV algorithm we prep ancilla in |-> to convert this flip into a phase.
          That makes 'b' become a global phase (i.e., doesn't change measured 's'). This is expected.
    """
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == 1:
        qc.x(ancilla)


# ============================================================
# 2) BV Algorithm Circuit (with optional constant b)
# ============================================================
def bernstein_vazirani_circuit(s: str, b: int = 0, measure_ancilla: bool = False):
    """
    Standard BV (phase-kickback) algorithm using |-> ancilla:
      - Prepare ancilla in |1>, then H -> |->.
      - H on all inputs -> uniform superposition.
      - Oracle flips ancilla based on f(x) (which becomes a phase on |->).
      - H on inputs -> reveals 's' deterministically.
    'b' only contributes a global phase, so it does NOT affect the measured input register.

    If measure_ancilla=True, we add one classical bit for ancilla and measure it,
    but in this BV form ancilla stays in |-> and measures ~50/50 in Z basis (not useful for 'b').
    """
    n = len(s)
    qreg = QuantumRegister(n + 1, "q")
    creg_in = ClassicalRegister(n, "c_in")
    if measure_ancilla:
        creg_a = ClassicalRegister(1, "c_a")
        qc = QuantumCircuit(qreg, creg_in, creg_a)
    else:
        qc = QuantumCircuit(qreg, creg_in)

    inputs = list(range(n))
    ancilla = n

    # Prepare |0...0> |1> then H on all to get |+...+>|->
    qc.x(ancilla)
    qc.h(qreg)

    # Oracle: computational flip on ancilla => phase kickback on |->
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    # Interference: H on input register to reveal s
    for q in inputs:
        qc.h(q)

    # Measure inputs (this is what recovers s)
    qc.measure(inputs, creg_in)

    # Optionally measure ancilla in Z basis (will be ~50/50, not informative about b in BV form)
    if measure_ancilla:
        qc.measure(ancilla, creg_a)

    return qc


# ============================================================
# 3) Runner (ideal simulator)
# ============================================================
def run_bv(qc, shots=1024, show_plot=True, title=None):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    if show_plot:
        if title:
            h2(title)
        plot_histogram(counts)
        plt.show()

    # Determine most frequent string on the input register width:
    n = qc.num_clbits if "c_a" not in [creg.name for creg in qc.cregs] else qc.cregs[0].size
    # If ancilla is also measured, counts keys may include a space "input ancilla".
    # Extract only the leftmost n bits (input measurement) from each key assuming format "input [space] ancilla".
    # Qiskit prints measured registers in order of declaration; we declared input creg first.
    most_key = max(counts, key=counts.get)
    # If space-separated, take first chunk (input)
    most_input = most_key.split(' ')[0]

    return counts, most_input


# ============================================================
# 4) Noise model and noisy runner
# ============================================================
def create_simple_noise_model():
    """
    A portable, simple depolarizing model:
    - 2% on single-qubit gates (x, h, sx, rz)
    - 5% on CX
    Works across Qiskit 2.x without relying on removed fake backends.
    """
    nm = NoiseModel()
    one_q = depolarizing_error(0.02, 1)
    two_q = depolarizing_error(0.05, 2)
    nm.add_all_qubit_quantum_error(one_q, ['x', 'h', 'sx', 'rz'])
    nm.add_all_qubit_quantum_error(two_q, ['cx'])
    return nm

def run_bv_noisy(qc, shots=4096, show_plot=True, title=None):
    noise_model = create_simple_noise_model()
    sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    if show_plot:
        if title:
            h2(title)
        plot_histogram(counts)
        plt.show()

    # Extract most common input string as above
    most_key = max(counts, key=count.get) if False else max(counts, key=counts.get)
    most_input = most_key.split(' ')[0]
    return counts, most_input


# ============================================================
# 5) IBM Runtime (Optional) — safe fallback
# ============================================================
def run_bv_ibm(s: str, b: int = 0, backend_name="ibmq_qasm_simulator", shots=4096):
    """
    Tries to sample on IBM Runtime SamplerV2. If unavailable, prints a helpful message.
    """
    h2(f"IBM Runtime run (backend: {backend_name})")
    try:
        if not IBM_RUNTIME_AVAILABLE:
            raise RuntimeError("qiskit_ibm_runtime not available in this environment.")

        qc = bernstein_vazirani_circuit(s, b=b, measure_ancilla=False)
        service = QiskitRuntimeService(channel="ibm_quantum")   # requires account linked
        backend = service.backend(backend_name)
        sampler = SamplerV2(backend)
        tqc = transpile(qc, backend)
        job = sampler.run([tqc], shots=shots)
        res = job.result()
        counts = res[0].data.meas.get_counts()

        plot_histogram(counts)
        plt.show()

        most_key = max(counts, key=counts.get)
        most_input = most_key.split(' ')[0]
        print("Most frequent input bits:", most_input)
        print("Expected s:", s)
        return counts, most_input
    except Exception as e:
        print("⚠ IBM run skipped:", e)
        print("Tip: link your account once:\n"
              "QiskitRuntimeService.save_account(channel='ibm_quantum', token='YOUR_TOKEN')\n"
              "Then restart the kernel and re-run.")
        return None, None


# ============================================================
# 6) Ancilla-only demonstration (computational oracle check)
# ============================================================
def ancilla_constant_demo(s: str, b: int):
    """
    Show that the 'b' term flips ONLY the ancilla in the computational-oracle sense.
    We do not prepare |->; we leave ancilla in |0> and also do not apply H to ancilla.
    With inputs = |0...0>, oracle computes ancilla -> b. We measure ancilla directly.
    This is not the BV algorithm; this is just to show the effect of 'b' in the oracle.
    """
    n = len(s)
    qreg = QuantumRegister(n + 1, "q")
    c_in = ClassicalRegister(n, "c_in")
    c_a  = ClassicalRegister(1, "c_a")
    qc = QuantumCircuit(qreg, c_in, c_a)

    inputs = list(range(n))
    ancilla = n
    # inputs start at |0...0>, ancilla at |0>
    # NO Hadamards at all here — we're just testing the oracle wiring.
    bv_oracle_with_b(qc, inputs, ancilla, s, b)
    # Measure ancilla; expect it to be 'b' because x=0 => s·x = 0.
    qc.measure(ancilla, c_a)
    # (We still include c_in to keep register order consistent, but we don't measure inputs)
    return qc


# ============================================================
# 7) Side-by-side plotting helper
# ============================================================
def compare_histograms(counts_a, label_a, counts_b, label_b, title=None):
    """
    Compare two distributions side-by-side. Qiskit's plot_histogram can overlay.
    """
    if title:
        h2(title)
    plot_histogram([counts_a, counts_b], legend=[label_a, label_b])
    plt.show()


# ============================================================
# 8) MAIN — Run the full set of tasks
# ============================================================
if __name__ == "__main__":
    h1("Bernstein–Vazirani Algorithm (Qiskit 2.x) — Full Demo")

    # -------- Task A: Change s and verify ----------
    h2("Task A: Change the secret string s and verify recovery")
    test_cases = [("1011", 0), ("0000", 1), ("11101", 0), ("01010", 1)]
    for s, b in test_cases:
        print(f"\nCase: s = {s}, b = {b}  (BV algorithm)")
        qc_bv = bernstein_vazirani_circuit(s, b=b, measure_ancilla=True)
        print(qc_bv.draw(fold=-1))
        counts_ideal, most_input_ideal = run_bv(qc_bv, shots=2048, show_plot=True, title=f"Ideal Simulator — s={s}, b={b}")
        print("Most frequent input bits:", most_input_ideal)
        print("Expected s:", s)
        if most_input_ideal == s:
            print("✅ Recovered s perfectly (as expected; b doesn't affect inputs).")
        else:
            print("❌ Mismatch — revisit your environment or inspect noise settings.")

        # -------- Task B: Show b affects ancilla only (in oracle sense) ----------
        h2("Task B: Demonstrate b toggles ancilla (computational-oracle check)")
        qc_b_only = ancilla_constant_demo(s, b)
        print(qc_b_only.draw(fold=-1))
        counts_cdemo, _ = run_bv(qc_b_only, shots=1024, show_plot=True, title=f"Ancilla-Only Oracle Check — s={s}, b={b}")
        print("NOTE: When x=0...0, oracle output on ancilla is just b. You should see the ancilla bit peaked at b.")
        print("Also note: In the actual BV algorithm, ancilla is in |-> so 'b' is a global phase and doesn't change input results.")

        # -------- Task C: Noise comparison ----------
        h2("Task C: Noise model — robustness comparison (Ideal vs Noisy)")
        counts_noisy, most_input_noisy = run_bv_noisy(qc_bv, shots=4096, show_plot=False)
        compare_histograms(counts_ideal, "Ideal", counts_noisy, "Noisy", title=f"Histogram Comparison — s={s}, b={b}")
        print("Noisy most frequent input bits:", most_input_noisy, " | Expected:", s)

    # -------- Optional: IBM Runtime ----------
    h2("Optional: IBM Runtime comparison")
    _counts_ibm, _most_ibm = run_bv_ibm("1011", b=0, backend_name="ibmq_qasm_simulator", shots=8192)

    # -------- Bonus: Oracle matrix for small n ----------
    h2("Bonus: Oracle matrix (small n demo)")
    s_small, b_small = "11", 1
    n = len(s_small)
    q = QuantumRegister(n + 1, "q")
    qc_oracle = QuantumCircuit(q)
    bv_oracle_with_b(qc_oracle, list(range(n)), n, s_small, b_small)
    print(qc_oracle.to_gate(label="U_f"))
    print("\nOracle matrix (computational-oracle action on ancilla):")
    print(Operator(qc_oracle).data)
    print("\nInterpretation:")
    print("- This gate flips the ancilla conditioned on the '1' positions in s, and toggles ancilla if b=1.")
    print("- In BV, because ancilla is prepared in |->, these controlled flips become phases, revealing 's' on the inputs.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.5 MB/s eta 0:00:00

Bernstein–Vazirani Algorithm (Qiskit 2.x) — Full Demo

Task A: Change the secret string s and verify recovery
------------------------------------------------------

Case: s = 1011, b = 0  (BV algorithm)
        ┌───┐          ┌───┐          ┌─┐           
   q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
        ├───┤┌───┐  │  └┬─┬┘          └╥┘           
   q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
        ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
   q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
        ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
   q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
        ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └┬─┬┘ ║ └╥┘
 